## Data Science Jobs in India - Naukri

<strong><a href="https://www.naukri.com/">Naukri.com</a></strong> is an Indian employment website operating in India and Middle East. Naukri.com was founded in March 1997. Naukri was ranked No.1 by 9 independent sources, placing it way ahead of competition.Google Trends names Naukri “the most preferred job search destination in India”.

In this project, I web scraped first 12,000 results obtained for <strong>'data scientist'</strong> job role in <strong>India</strong>

In [1]:
# pip install selenium

In [2]:
# import libraries

import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

import warnings
warnings.filterwarnings('ignore')

In [3]:
# webdriver for Firefox - geckodriver
driver = webdriver.Firefox(executable_path='/home/futures/Documents/geckodriver')

# sticky timeout to implicitly wait for an element to be found
driver.implicitly_wait(10) 

# open url in the browser
driver.get('https://www.naukri.com/')

In [4]:
# enter job role - data scientist
role=driver.find_element_by_xpath('/html/body/div/div[2]/div[3]/div/div/div[1]/div/div/div/input')
role.send_keys('data scientist')

# location - India
location=driver.find_element_by_xpath('/html/body/div/div[2]/div[3]/div/div/div[3]/div/div/div/input')
location.send_keys('india')

# click search
driver.find_element_by_class_name('qsbSubmit').click()

In [6]:
# new dataframe fwith required columns
df=pd.DataFrame(columns=['Job_Role','Company','Location','Job Experience','Skills/Description'])
df

,Job_Role,Company,Location,Job Experience,Skills/Description


In [7]:
# helper function

# Function to convert  a list to string
def listToString(s): 
    
    # initialize an empty string
    str1 = ", " 
    
    # return string  
    return (str1.join(s))

In [8]:
%%time

# page number
page = 1

condition = True

while condition:

    # atmost 20 results per page
    for i in range(1,21):

        try:

            role = driver.find_element_by_xpath('/html/body/div/div[3]/div[2]/section[2]/div[2]/article[{}]/div[1]/div[1]/a'.format(i)).text
            company = driver.find_element_by_xpath('/html/body/div/div[3]/div[2]/section[2]/div[2]/article[{}]/div[1]/div[1]/div/a[1]'.format(i)).text
            location = driver.find_element_by_xpath('/html/body/div/div[3]/div[2]/section[2]/div[2]/article[{}]/div[1]/div[1]/ul/li[3]/span[1]'.format(i)).text
            experience = driver.find_element_by_xpath('/html/body/div/div[3]/div[2]/section[2]/div[2]/article[{}]/div[1]/div/ul/li[1]/span'.format(i)).text.replace(' Yrs','')
            skill_lst = driver.find_element_by_xpath('/html/body/div/div[3]/div[2]/section[2]/div[2]/article[{}]/ul'.format(i)).text.split('\n')
            skill = listToString(skill_lst)

            # append collected information to the dataframe df
            df.loc[len(df.index)] = [role,company,location,experience,skill]


        except NoSuchElementException:

            pass

    page += 1

    # click next
    nxt=driver.find_element_by_xpath('/html/body/div/div[3]/div[2]/section[2]/div[3]/div/a[2]')
    driver.execute_script("arguments[0].click();", nxt)

    # total 12700+ results showed, 20 results per page => 600+ pages

    # stop after 620 pages
    if page >= 620:
        condition = False

CPU times: user 1min 14s, sys: 2.19 s, total: 1min 16s
Wall time: 18min 19s


In [11]:
# drop duplicates
df=df.drop_duplicates()

# shape of the dataframe
df.shape

(12163, 5)

In [12]:
# selecting first 12,000 results
df=df.iloc[:12000,:]
df.shape

(12000, 5)

In [13]:
# first five rows
df.head()

,Job_Role,Company,Location,Job Experience,Skills/Description
0,Data Scientist: Artificial Intelligence,IBM,Bangalore/Bengaluru,6-11,"Scheduling, big data, Unix shell scripting, SQ..."
1,Staff Data Scientist,Walmart,Bangalore/Bengaluru,4-8,"IT Skills, Data Science, Machine Learning, Big..."
2,Data Scientist-Client Engineering,IBM,Mumbai,5-10,"Consulting, Machine learning, Healthcare, Pres..."
3,Senior Data Scientist,Walmart,Bangalore/Bengaluru,5-10,"IT Skills, Testing, Data Science, Machine Lear..."
4,Cognitive/AI Senior Data Scientist,IBM,Bangalore/Bengaluru,4-8,"Automation, Consulting, Machine learning, Anal..."


In [14]:
# last five rows
df.tail()

,Job_Role,Company,Location,Job Experience,Skills/Description
12208,Cognos Developer,Datametica Solutions Pvt Ltd,"Hyderabad/Secunderabad, Pune",4-9,"Cognos Framework Manager, Cognos, Automation, ..."
12209,"Program Manager (IT - Retail Industry), 30-35 ...",Talent Bee Consulting Pvt. Ltd,Bangalore/Bengaluru,10-15,"IT Skills, Java, Software Development, Data Sc..."
12210,"Java Developer - 3-6 Years, 24 LPA, Bangalore,...",Talent Bee Consulting Pvt. Ltd,Bangalore/Bengaluru,3-6,"MySQL, PostgresSQL, REST, OO, J2ee, patterns, ..."
12211,Node.JS Developer,Sysvine Technologies,Remote,2-7,"Javascript, Node.Js, React.Js, Angular JS, Vue..."
12212,Developer,Orcapod Consulting Services Private Limited,Bangalore/Bengaluru,4-8,"ES6, Enzyme, LESS, Jest, CSS, Highcharts, D3, ..."


In [15]:
# save as excel file
df.to_excel('naukri_data_science_jobs_india.xlsx',index=False)